<span style="color:#888888">Copyright (c) 2014-2021 National Technology and Engineering Solutions of Sandia, LLC. Under the terms of Contract DE-NA0003525 with National Technology and Engineering Solutions of Sandia, LLC, the U.S. Government retains certain rights in this software.     Redistribution and use in source and binary forms, with or without modification, are permitted provided that the following conditions are met:</span>

<span style="color:#888888">1. Redistributions of source code must retain the above copyright notice, this list of conditions and the following disclaimer.</span>

<span style="color:#888888">2. Redistributions in binary form must reproduce the above copyright notice, this list of conditions and the following disclaimer in the documentation and/or other materials provided with the distribution.</span>

<span style="color:#888888">THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.</span>

# <span style="color:#0054a8">**Demo 5:**</span> <span style="color:#555555">Clustering Trajectories by Shape</span>

Using Tracktable's box-DBSCAN capabilities, we can detect similarly shaped trajectories regardless of location or scale.

<details closed>
    <summary><b>Algorithm Details</span></b></summary>
    <br>
    For every trajectory, we create a <span style="color:#00add0"><b>feature vector</b></span>, which we call the distance geometry feature vector, as follows:
    <ul>
        <li><i>1st value:</i> Take the distance between the endpoints of the trajectory (as the crow flies) and divide by the total distance traveled, giving us a measure of "straightness".</li>
        <li><i>2nd and 3rd values:</i> Find the midpoint along the trajectory such that it splits the distance traveled along the trajectory in half.  Do the same calculation again, but for each half.  That is, take the distance between the start point and the midpoint of the trajectory (as the crow flies) and divide by the distance traveled along the trajectory from the start point to the midpoint.  Save that as the second value in the feature vector.  Then take the distance between the midpoint and the endpoint and divide by the distance traveled along the trajectory between those points.  This is the third value in the feature vector.</li>
        <li><i>4th, 5th and 6th values:</i> Do the same calculation, but with thirds.</li>
        <li><i>7th, 8th, 9th and 10th values:</i> Do the same calculation, but with fourths.</li>
    </ul>
    ... and so on.  We stop when we reach a preset "depth" which is the largest number of segments that we divide our trajectory into.  For instance, a depth of 4 will yield a 10-dimensional feature vector.  In this way, the distance geometry feature vector encapsulates the straightness of the trajectory across proportionally smaller and smaller segments, and thus gives a concise and comparable representation of shape for the trajectory.  This representation is scale, rotation, translation and reflection <span style="color:#00add0"><b>invariant</b></span>.
    <br>
    <br>
    Using <span style="color:#00add0"><b>box-DBSCAN</b></span>, Tracktable clusters trajectories together based on the similarity of their feature vectors.  Trajectories that are similar in shape should occupy the same cluster.
</details>

In [ ]:
from tracktable.render.render_trajectories import render_trajectories_separate, render_trajectories
import tracktable.examples.tutorials.tutorial_helper as tutorial 
from tracktable.applications.cluster import *

## Import Trajectories

We will use some sample maritime data for this demo, obtained from BOEM/NOAA.$^1$

In [ ]:
trajectories = tutorial.get_trajectory_list('shape')

## Trajectory Clustering using Distance Geometry Feature Vectors

Create a distance geometry feature vector (as defined in "Algorithm Details" above) for each trajectory and use Tracktable's box-DBSCAN to cluster the trajectories.

The `epsilon` parameter defines inradius of the "nearness" box for box-DBSCAN.  Increasing (decreasing) it will result in a greater (smaller) likelihood that trajectories will cluster together, reducing (increasing) the number of outliers.

In [ ]:
clusters = cluster_trajectories_shape(trajectories, depth=4, epsilon=0.02)

The `clusters` dictionary contains cluster numbers as keys, and a list of trajectories in that cluster for each corresponding value.  These clusters and their sizes are printed above.

In our trajectory dataset, we have found 5 clusters, meaning 5 underlying shapes were prevalent in our dataset of 125 maritime trajectories.

## Cluster Visualization

Let's look at what the trajectories in some of these clusters look like.

### <span style="color:#0f0f0f">*Cluster 3: Trajectories that double-back and return to their origin.*</span>

In [ ]:
render_trajectories(clusters[3])

In [ ]:
render_trajectories_separate(clusters[3])

### <span style="color:#0f0f0f">*Cluster 4: Trajectories with a sharp turn at one end.*</span>

In [ ]:
render_trajectories(clusters[4])

In [ ]:
render_trajectories_separate(clusters[4])

### <span style="color:#0f0f0f">*Cluster 7: Trajectories that are ferrying in a mostly straight line.*</span>

In [ ]:
render_trajectories(clusters[7])

In [ ]:
render_trajectories_separate(clusters[7])

### <span style="color:#0f0f0f">*Cluster 8: Trajectories that are ferrying with sharp turns.*</span>

In [ ]:
render_trajectories(clusters[8])

In [ ]:
render_trajectories_separate(clusters[8])

## Outlier Visualization

The trajectories that did not cluster (outliers) are stored under key `0` of the `clusters` dictionary.  Let's look at the first five of these anomalously shaped trajectories.

In [ ]:
render_trajectories_separate(clusters[0][:5])

<span style="color:gray">$^1$ Bureau of Ocean Energy Management (BOEM) and National Oceanic and Atmospheric Administration (NOAA). MarineCadastre.gov. *AIS Data for 2020.* Retrieved February 2021 from [marinecadastre.gov/data](https://marinecadastre.gov/data/).  US coastal maritime traffic trimmed down to June 30, 2020.</span>